In [11]:
from dataset import get_datasets
_, _, text_vect= get_datasets()

2024-01-12 16:53:05.905577: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [9248]
	 [[{{node Placeholder/_1}}]]
2024-01-12 16:53:05.905719: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [9248]
	 [[{{node Placeholder/_1}}]]
2024-01-12 16:53:06.147392: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [9248

In [15]:
text_vect([''])[0]

<tf.Tensor: shape=(0,), dtype=int64, numpy=array([], dtype=int64)>

In [2]:
import tensorflow as tf

In [3]:
model = tf.keras.Sequential([text_vect])
model.save("text_processor")

INFO:tensorflow:Assets written to: text_processor/assets


In [5]:
text_vect = tf.keras.models.load_model("text_processor").layers[0]

In [6]:
text_vect

In [1]:
import tensorflow as tf

@tf.keras.utils.register_keras_serializable(name="text_standardization")
def tf_lower_and_split_punct(text):
    "Text standardization function. Tries to make things uniform."
    text = tf.strings.lower(text) # Lowercase everything
    text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '') # Keep space, a to z and punctuation.
    text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ') # Add spaces around punctuation.
    text = tf.strings.strip(text) # Strip whitespace.

    text = tf.strings.join(['[START]', text, '[END]'], separator=' ') # Add start and end token
    return text

2024-01-12 16:46:41.703967: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-12 16:46:41.726890: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-12 16:46:42.193767: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
from inference import ScratchModel
from IPython.display import clear_output

model = ScratchModel()

In [2]:
question = "How can I apply to SUNY Brockport?"

generator = model(question, stream=True)

for token in generator:
    print(token)
    clear_output(wait=True)

InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:GPU:0}} slice index 0 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/

In [2]:
import tensorflow as tf

In [23]:
loaded_model = tf.keras.models.load_model("./models/transformer_v4/" + "text_processor")

# Retrieve the TextVectorization layer
loaded_text_processor = loaded_model.layers[0]

In [1]:
import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # stop showing tensorflow logs
from datasets import load_dataset
import tensorflow as tf
import numpy as np

seed = 9
np.random.seed(seed)
tf.random.set_seed(seed)

dataset = load_dataset("msaad02/brockport-gpt-4-qa")
dataset = dataset['train'].to_pandas()

# Note that the prompt is never used in the dataset, but it is included 
# here as an artifact from the original implementation of the scratch model
prompt = lambda question, answer: f"""Below is an inquiery related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.\n\n### Instruction:\n{question}\n\n### Response:\n{answer}"""

dataset = [prompt(question, answer) for question, answer in zip(dataset['question'], dataset['answer'])]

def split_input_output(s):
    """
    Splits a string into a question and answer pair
    """
    output_split = s.split('\n\n### Response:\n')
    input_split = output_split[0].split('### Instruction:\n')[1]
    return input_split, output_split[1]

context_raw, target_raw = [list(t) for t in zip(*[split_input_output(string) for string in dataset])]

# Add start and end tokens
target_raw = ["[START] " + text + " [END]" for text in target_raw]

BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 64

is_train_mask = np.random.uniform(size=(len(target_raw),)) < 0.8

train_context = np.array(context_raw)[is_train_mask]
train_target = np.array(target_raw)[is_train_mask]

val_context = np.array(context_raw)[~is_train_mask]
val_target = np.array(target_raw)[~is_train_mask]

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((train_context, train_target))
    .shuffle(BUFFER_SIZE, seed=seed)
    .batch(BATCH_SIZE)
)
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((val_context, val_target))
    .shuffle(BUFFER_SIZE, seed=seed)
    .batch(BATCH_SIZE)
)

MAX_VOCAB_SIZE = 5000

text_processor = tf.keras.layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=MAX_VOCAB_SIZE,
    ragged=True
)

text_processor.adapt(train_raw.map(lambda context, target: context))
text_processor.adapt(train_raw.map(lambda context, target: target))

def process_text(context, target):
    context = text_processor(context).to_tensor()
    target  = text_processor(target)
    targ_in = target[:,:-1].to_tensor()
    targ_out = target[:,1:].to_tensor()
    return (context, targ_in), targ_out

train_ds = train_raw.map(process_text, tf.data.AUTOTUNE)
val_ds = val_raw.map(process_text, tf.data.AUTOTUNE)


2024-01-12 16:59:47.879694: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-12 16:59:47.902993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-12 16:59:48.377749: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-01-12 16:59:49.889080: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-12 16:59:49.900

In [4]:
for batch in val_ds.take(1):  # Adjust the number to how many batches you want to inspect
    # Assuming each batch contains a tuple of (inputs, targets)
    inputs, targets = batch
    print("Inputs:", inputs)
    print("Targets:", targets)
    print("-----")


Inputs: (<tf.Tensor: shape=(64, 23), dtype=int64, numpy=
array([[ 282,  884,    3, ...,    0,    0,    0],
       [ 305,   22,    3, ...,    0,    0,    0],
       [ 138,  884,    3, ...,    0,    0,    0],
       ...,
       [  50, 2262, 4346, ...,    0,    0,    0],
       [ 138,  370,  789, ...,    0,    0,    0],
       [  22,  103,   49, ...,   30,    1,  349]])>, <tf.Tensor: shape=(64, 82), dtype=int64, numpy=
array([[   7,    3,  647, ...,    0,    0,    0],
       [   7,    9,   97, ...,    0,    0,    0],
       [   7,    3, 1629, ...,    0,    0,    0],
       ...,
       [   7,  208,    9, ...,    0,    0,    0],
       [   7,    2,  161, ...,   57,    3,   66],
       [   7,  127,  103, ...,    0,    0,    0]])>)
Targets: tf.Tensor(
[[   3  647 1029 ...    0    0    0]
 [   9   97   56 ...    0    0    0]
 [   3 1629  240 ...    0    0    0]
 ...
 [ 208    9 4346 ...    0    0    0]
 [   2  161    6 ...    3   66    8]
 [ 127  103   22 ...    0    0    0]], shape=(64, 82), 

2024-01-12 17:02:54.384447: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype resource
	 [[{{node Placeholder/_11}}]]
2024-01-12 17:02:54.384641: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype resource
	 [[{{node Placeholder/_11}}]]


In [13]:
targets[]

<tf.Tensor: shape=(82,), dtype=int64, numpy=
array([   3,  647, 1029,   10,  205,   23, 2170,   12,  657,    4,    1,
         10,    6,  441,  130, 1375,    3,  383,  657,   31,   38, 2208,
          3,  104,  141,    3,  364,  102,    9, 1029,   50, 1922,   41,
       1345,    2,  657,    4,   10,  294,  732,   12,    3,  288, 1029,
         14,   41,  112,    2,  657,   26,   91,    2,  184,    5,  868,
        423,    9,   37,  159,    8,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0])>

In [14]:
"""
This script creates a tensorflow dataset for the brockport-gpt-4-qa dataset to feed into the model.

This also does some preprocessing of the data, such as adding start and end tokens, and standardizing the text.
"""

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # stop showing tensorflow logs
from datasets import load_dataset
import tensorflow as tf
import numpy as np

seed = 9
np.random.seed(seed)
tf.random.set_seed(seed)

dataset = load_dataset("msaad02/brockport-gpt-4-qa")
dataset = dataset['train'].to_pandas()

# Note that the prompt is never used in the dataset, but it is included 
# here as an artifact from the original implementation of the scratch model
prompt = lambda question, answer: f"""Below is an inquiery related to SUNY Brockport - from academics, admissions, and faculty support to student life. Prioritize accuracy and brevity.\n\n### Instruction:\n{question}\n\n### Response:\n{answer}"""

dataset = [prompt(question, answer) for question, answer in zip(dataset['question'], dataset['answer'])]

def split_input_output(s):
    """
    Splits a string into a question and answer pair
    """
    output_split = s.split('\n\n### Response:\n')
    input_split = output_split[0].split('### Instruction:\n')[1]
    return input_split, output_split[1]

# @tf.keras.saving.register_keras_serializable(name="text_standardization")
@tf.keras.utils.register_keras_serializable(name="text_standardization")
def tf_lower_and_split_punct(text):
    "Text standardization function. Tries to make things uniform."
    text = tf.strings.lower(text) # Lowercase everything
    text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '') # Keep space, a to z and punctuation.
    text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ') # Add spaces around punctuation.
    text = tf.strings.strip(text) # Strip whitespace.

    text = tf.strings.join(['[START]', text, '[END]'], separator=' ') # Add start and end token
    return text

context_raw, target_raw = [list(t) for t in zip(*[split_input_output(string) for string in dataset])]

BUFFER_SIZE = len(context_raw)
BATCH_SIZE = 64

is_train_mask = np.random.uniform(size=(len(target_raw),)) < 0.8

train_context = np.array(context_raw)[is_train_mask]
train_target = np.array(target_raw)[is_train_mask]

val_context = np.array(context_raw)[~is_train_mask]
val_target = np.array(target_raw)[~is_train_mask]

train_raw = (
    tf.data.Dataset
    .from_tensor_slices((train_context, train_target))
    .shuffle(BUFFER_SIZE, seed=seed)
    .batch(BATCH_SIZE)
)
val_raw = (
    tf.data.Dataset
    .from_tensor_slices((val_context, val_target))
    .shuffle(BUFFER_SIZE, seed=seed)
    .batch(BATCH_SIZE)
)

MAX_VOCAB_SIZE = 5000

text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=MAX_VOCAB_SIZE,
    ragged=True
)

text_processor.adapt(train_raw.map(lambda context, target: context))
text_processor.adapt(train_raw.map(lambda context, target: target))

def process_text(context, target):
    context = text_processor(context).to_tensor()
    target  = text_processor(target)
    targ_in = target[:,:-1].to_tensor()
    targ_out = target[:,1:].to_tensor()
    return (context, targ_in), targ_out

train_ds = train_raw.map(process_text, tf.data.AUTOTUNE)
val_ds = val_raw.map(process_text, tf.data.AUTOTUNE)


2024-01-12 17:06:11.993472: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [9248]
	 [[{{node Placeholder/_1}}]]
2024-01-12 17:06:11.993611: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [9248]
	 [[{{node Placeholder/_1}}]]
2024-01-12 17:06:12.278709: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype string and shape [9248

In [18]:
for batch in val_ds.take(3):  # Adjust the number to how many batches you want to inspect
    # Assuming each batch contains a tuple of (inputs, targets)
    inputs, targets = batch
    print("Inputs:", inputs)
    print("Targets:", targets)
    print("-----")


Inputs: (<tf.Tensor: shape=(64, 27), dtype=int64, numpy=
array([[   9,  286,  888, ...,    0,    0,    0],
       [   9,  310,   25, ...,    0,    0,    0],
       [   9,  141,  888, ...,    0,    0,    0],
       ...,
       [   9,   53, 2236, ...,    0,    0,    0],
       [   9,  141,  374, ...,    0,    0,    0],
       [   9,   25,  106, ...,  354, 1568,   10]])>, <tf.Tensor: shape=(64, 92), dtype=int64, numpy=
array([[   9,    5,  647, ...,    0,    0,    0],
       [   9,   11,  100, ...,    0,    0,    0],
       [   9,    5, 1621, ...,    0,    0,    0],
       ...,
       [   9,  212,    2, ...,    0,    0,    0],
       [   9,    4,  164, ...,    5,   69,   15],
       [   9,  129,    2, ...,    0,    0,    0]])>)
Targets: tf.Tensor(
[[   5  647 1031 ...    0    0    0]
 [  11  100   59 ...    0    0    0]
 [   5 1621  244 ...    0    0    0]
 ...
 [ 212    2   11 ...    0    0    0]
 [   4  164    8 ...   69   15   10]
 [ 129    2  106 ...    0    0    0]], shape=(64, 92), 

2024-01-12 17:06:38.487070: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [2334]
	 [[{{node Placeholder/_0}}]]
2024-01-12 17:06:38.487306: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype resource
	 [[{{node Placeholder/_11}}]]


In [19]:
text_processor([''])

<tf.RaggedTensor [[9, 10]]>

In [22]:
inputs[0][0]

<tf.Tensor: shape=(24,), dtype=int64, numpy=
array([   9,  141,   19,  716,  214,    5, 2760, 4156, 1266,   13,  776,
       1280, 2760,   20,   23,   21, 1568,   10,    0,    0,    0,    0,
          0,    0])>